In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
chars = "abcdefghijklmnopqrstuvwxyz .,:;?01" #0:start code, 1:end code
char_list = [i for i in chars]
n_letters = len(char_list)

In [ ]:
def word_to_onehot(string):
  start=np.zeros(shape=len(char_list), dtype=int)
  end=np.zeros(shape=len(char_list), dtype=int)
  start[-2]=1
  end[-1]=1

  for i in string:
    idx=char_list.index(i)
    zero=np.zeros(shape=n_letters, dtype=int)
    zero[idx]=1
    start=np.vstack([start, zero])
  output=np.vstack([start, end])
  return output

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

class myRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
      super(myRNN,  self).__init__()

      self.input_size = input_size #input size=글자수
      self.hidden_size = hidden_size #hidden layer와 관련돼 맘대로 설정.
      self.output_size=output_size

      self.U=nn.Linear(input_size, hidden_size)
      self.W=nn.Linear(hidden_size, hidden_size)
      self.V=nn.Linear(hidden_size, output_size)
      self.activation_func=nn.Tanh()
      self.softmax=nn.LogSoftmax(dim=0)

    def forward(self, input, hidden):
      hidden=self.activation_func(self.U(input)+self.W(hidden))
      output=self.V(hidden)
      return output,hidden

    def init_hidden(self):
      return torch.zeros(1, self.hidden_size)

In [ ]:
def main():
  n_hidden=128
  lr=0.001
  epochs=1000
  string="i want to go on a trip these days. how about you? "
  #string="i want to go on a trip these days. how about you? i would like to visit france, italy, germany again with my friends."

  rnn=myRNN(n_letters, n_hidden, n_letters)

  loss_func=nn.CrossEntropyLoss()
  optimizer=torch.optim.Adam(rnn.parameters(), lr=lr)

  one_hot=torch.from_numpy(word_to_onehot(string)).type_as(torch.FloatTensor())

  for i in range(epochs):
    rnn.zero_grad() #CNN과의 차이점: RNN은 1epoch마다 zerograd해줌. ㅊㅊㅊㅊㅊㅊ
    total_loss=0
    hidden=rnn.init_hidden()

    input=one_hot[0:1,:]
    for j in range(one_hot.size()[0]-1):
      target=one_hot[j+1]
      target_single=torch.from_numpy(np.asarray(target.numpy().argmax())).type_as(torch.LongTensor()).view(-1)

      output, hidden=rnn.forward(input, hidden)
      loss=loss_func(output, target_single)
      total_loss+=loss
      input=output
    total_loss.backward()
    optimizer.step()

    if i%10==0:
      print('epoch%d'%i)
      print(total_loss)

  start=torch.zeros(1, len(char_list))
  start[:,-1]=1

  print('Origin: ',string)
  print('Result:  ',end='')
  with torch.no_grad():
    hidden=rnn.init_hidden()
    input=start
    output_string=""
    for i in range(len(string)):
      output, hidden=rnn.forward(input, hidden)
      output_string+=onehot_to_word(output.data)
      input=output
    print(output_string)



if __name__ == '__main__':
    main()

epoch0
tensor(179.4292, grad_fn=<AddBackward0>)
epoch10
tensor(148.5619, grad_fn=<AddBackward0>)
epoch20
tensor(138.7971, grad_fn=<AddBackward0>)
epoch30
tensor(135.9217, grad_fn=<AddBackward0>)
epoch40
tensor(134.1267, grad_fn=<AddBackward0>)
epoch50
tensor(131.7123, grad_fn=<AddBackward0>)
epoch60
tensor(128.2314, grad_fn=<AddBackward0>)
epoch70
tensor(124.6018, grad_fn=<AddBackward0>)
epoch80
tensor(121.4894, grad_fn=<AddBackward0>)
epoch90
tensor(118.9099, grad_fn=<AddBackward0>)
epoch100
tensor(116.5424, grad_fn=<AddBackward0>)
epoch110
tensor(113.3726, grad_fn=<AddBackward0>)
epoch120
tensor(117.0034, grad_fn=<AddBackward0>)
epoch130
tensor(115.1845, grad_fn=<AddBackward0>)
epoch140
tensor(113.1740, grad_fn=<AddBackward0>)
epoch150
tensor(109.5175, grad_fn=<AddBackward0>)
epoch160
tensor(104.6153, grad_fn=<AddBackward0>)
epoch170
tensor(100.2840, grad_fn=<AddBackward0>)
epoch180
tensor(96.8132, grad_fn=<AddBackward0>)
epoch190
tensor(92.7745, grad_fn=<AddBackward0>)
epoch200
tens